In [1]:
import requests
import json
import pandas as pd
import time
import random

In [2]:
# Initialize an empty list to store each batch of matches data
all_matches = []

# Loop over drillDownTagIds ranges in steps of 1000
for start in range(0, 20000, 1000):
    # Generate a comma-separated string of drillDownTagIds for the current range
    drilldown_ids = ','.join(str(i) for i in range(start, start + 1000))

    # Define the request URL
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/event-list?"
        "startTimeFrom=2024-12-10T23:00:00Z&"
        "startTimeTo=2024-11-21T22:59:59Z&"
        "liveNow=false&"
        "maxEvents=190&"
        "orderEventsBy=popularity&"
        "orderMarketsBy=displayOrder&"
        "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
        "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
        "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
        "eventSortsIncluded=MTCH&"
        "includeChildMarkets=true&"
        "prioritisePrimaryMarkets=true&"
        "includeCommentary=true&"
        "includeMedia=true&"
        f"drilldownTagIds={drilldown_ids}&"
        "categoryCode=tennis&"  # Specify the desired category
        "lang=nl-NL&"
        "channel=I"
    )

    # Headers for the request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    }

    # Make the request
    response = requests.get(url, headers=headers)

    # Parse the JSON response
    data = response.json()

    # Extract match data if available
    matches = []
    for event in data.get('data', {}).get('events', []):
        match = {
            "event_id": event.get("id"),
            "match_name": event.get("name"),
            "start_time": event.get("startTime"),
            "home_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "HOME"), None),
            "away_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "AWAY"), None),
            "competition": event.get('type', {}).get('name'),
            "country": event.get('class', {}).get('name'),
            "sport": event.get('category', {}).get('name'),
        }
        
        # Extract odds if available
        outcomes = event.get("markets", [{}])[0].get("outcomes", [])
        match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "H"), None)
        match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "A"), None)
        
        matches.append(match)

    # Add the batch of matches to the all_matches list
    all_matches.extend(matches)

    # Sleep for a random duration between 1 and 4 seconds, with variability of 0.1 ms
    time.sleep(random.uniform(1, 2.001))

# Combine all match data into a single DataFrame
final_matches_df = pd.DataFrame(all_matches)

# Display the resulting DataFrame
final_matches_df

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
0,5699848,Frankrijk vs Israël,2024-11-14T19:45:00Z,Frankrijk,Israël,UEFA Nations League,Europees,Voetbal,1.13,20.00
1,5699853,Kazachstan vs Oostenrijk,2024-11-14T15:00:00Z,Kazachstan,Oostenrijk,UEFA Nations League,Europees,Voetbal,10.50,1.27
2,5699842,Portugal vs Polen,2024-11-15T19:45:00Z,Portugal,Polen,UEFA Nations League,Europees,Voetbal,1.26,11.00
3,5699855,Griekenland vs Engeland,2024-11-14T19:45:00Z,Griekenland,Engeland,UEFA Nations League,Europees,Voetbal,4.25,1.88
4,5699864,Nederland vs Hongarije,2024-11-16T19:45:00Z,Nederland,Hongarije,UEFA Nations League,Europees,Voetbal,1.39,7.75
...,...,...,...,...,...,...,...,...,...,...
1348,5955050,Radek Rose vs Zdenek Bilek,2024-11-12T16:30:00Z,Radek Rose,Zdenek Bilek,Czech Liga Pro,Tsjechië,Tafeltennis,2.45,1.49
1349,5955064,Radim Pavelka vs Frantisek Briza,2024-11-12T19:00:00Z,Radim Pavelka,Frantisek Briza,Czech Liga Pro,Tsjechië,Tafeltennis,1.53,2.35
1350,5955055,Radim Pavelka vs Martin Sychra,2024-11-12T21:00:00Z,Radim Pavelka,Martin Sychra,Czech Liga Pro,Tsjechië,Tafeltennis,1.65,2.10
1351,5927585,Radnicki Kraguljevac vs RK Vranje 1957,2024-11-12T18:00:00Z,Radnicki Kraguljevac,RK Vranje 1957,Serbia Superliga,Servië,Handbal,1.06,10.00


In [3]:
final_matches_df

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
0,5699848,Frankrijk vs Israël,2024-11-14T19:45:00Z,Frankrijk,Israël,UEFA Nations League,Europees,Voetbal,1.13,20.00
1,5699853,Kazachstan vs Oostenrijk,2024-11-14T15:00:00Z,Kazachstan,Oostenrijk,UEFA Nations League,Europees,Voetbal,10.50,1.27
2,5699842,Portugal vs Polen,2024-11-15T19:45:00Z,Portugal,Polen,UEFA Nations League,Europees,Voetbal,1.26,11.00
3,5699855,Griekenland vs Engeland,2024-11-14T19:45:00Z,Griekenland,Engeland,UEFA Nations League,Europees,Voetbal,4.25,1.88
4,5699864,Nederland vs Hongarije,2024-11-16T19:45:00Z,Nederland,Hongarije,UEFA Nations League,Europees,Voetbal,1.39,7.75
...,...,...,...,...,...,...,...,...,...,...
1348,5955050,Radek Rose vs Zdenek Bilek,2024-11-12T16:30:00Z,Radek Rose,Zdenek Bilek,Czech Liga Pro,Tsjechië,Tafeltennis,2.45,1.49
1349,5955064,Radim Pavelka vs Frantisek Briza,2024-11-12T19:00:00Z,Radim Pavelka,Frantisek Briza,Czech Liga Pro,Tsjechië,Tafeltennis,1.53,2.35
1350,5955055,Radim Pavelka vs Martin Sychra,2024-11-12T21:00:00Z,Radim Pavelka,Martin Sychra,Czech Liga Pro,Tsjechië,Tafeltennis,1.65,2.10
1351,5927585,Radnicki Kraguljevac vs RK Vranje 1957,2024-11-12T18:00:00Z,Radnicki Kraguljevac,RK Vranje 1957,Serbia Superliga,Servië,Handbal,1.06,10.00


In [4]:
# import requests

# # Generate a comma-separated string of drillDownTagIds between 11000 and 12999
# drilldown_ids = ','.join(str(i) for i in range(0, 1000))

# url = (
#     f"https://content.toto.nl/content-service/api/v1/q/event-list?"
#     "startTimeFrom=2024-11-10T23:00:00Z&"
#     "startTimeTo=2024-11-21T22:59:59Z&"
#     "liveNow=false&"
#     "maxEvents=250&"
#     "orderEventsBy=popularity&"
#     "orderMarketsBy=displayOrder&"
#     "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
#     "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
#     "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
#     "eventSortsIncluded=MTCH&"
#     "includeChildMarkets=true&"
#     "prioritisePrimaryMarkets=true&"
#     "includeCommentary=true&"
#     "includeMedia=true&"
#     f"drilldownTagIds={drilldown_ids}&"  # Generated drilldown tag IDs
#     "categoryCode=tennis&"  # Specify the desired category
#     "lang=nl-NL&"
#     "channel=I"
# )

# headers = {
#     'accept': 'application/json',
#     'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
# }

# response = requests.get(url, headers=headers)

# # Parsing the response
# data = response.json()

# # Display the JSON structure for verification
# print(data)

In [5]:
# # Initialize an empty list to collect match data
# matches = []

# # Extract match details from each event
# for event in data['data']['events']:
#     match = {
#         "event_id": event.get("id"),
#         "match_name": event.get("name"),
#         "start_time": event.get("startTime"),
#         "home_team": next((team['name'] for team in event['teams'] if team['side'] == "HOME"), None),
#         "away_team": next((team['name'] for team in event['teams'] if team['side'] == "AWAY"), None),
#         "competition": event['type']['name'],
#         "country": event['class']['name'],
#         "sport": event['category']['name'],
#     }
    
#     # Extract odds, if available
#     outcomes = event.get("markets", [{}])[0].get("outcomes", [])
#     if outcomes:
#         match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome['subType'] == "H"), None)
#         match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome['subType'] == "A"), None)
    
#     matches.append(match)

# # Convert the match list to a DataFrame
# matches_df = pd.DataFrame(matches)

# # Display the resulting DataFrame
# matches_df

In [6]:
# url = "https://content.toto.nl/content-service/api/v1/q/popular-bets-event-list?minPrice=1.25&maxPrice=9.90&combinable=true&numOutcomes=3&startTimeFrom=2024-11-09T13%3A32%3A35Z&startTimeTo=2024-12-10T13%3A32%3A35Z&marketSortsIncluded=HH%2CMR&lang=nl-NL&channel=I"

# payload = {}
# headers = {
#   'accept': 'application/json',
#   'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
# }

# response = requests.request("GET", url, headers=headers, data=payload)

# match_data = json.loads(response.text)


In [7]:
# # Initialize an empty list to collect match data
# matches = []

# # Extract match details from each event
# for event in match_data['data']['events']:
#     match = {
#         "event_id": event.get("id"),
#         "match_name": event.get("name"),
#         "start_time": event.get("startTime"),
#         "home_team": next((team['name'] for team in event['teams'] if team['side'] == "HOME"), None),
#         "away_team": next((team['name'] for team in event['teams'] if team['side'] == "AWAY"), None),
#         "competition": event['type']['name'],
#         "country": event['class']['name'],
#         "sport": event['category']['name'],
#     }
    
#     # Extract odds, if available
#     outcomes = event.get("markets", [{}])[0].get("outcomes", [])
#     if outcomes:
#         match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome['subType'] == "H"), None)
#         match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome['subType'] == "A"), None)
    
#     matches.append(match)

# # Convert the match list to a DataFrame
# matches_df = pd.DataFrame(matches)

# # Display the resulting DataFrame
# matches_df

In [8]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to accumulate all data
all_data = []

# Set batch size
batch_size = 100

# Loop over event_ids in batches of 500
for i in range(0, len(final_matches_df['event_id']), batch_size):
    # Get a batch of event_ids
    batch_ids = final_matches_df['event_id'][i:i + batch_size]
    
    # Concatenate event_ids for the URL
    event_ids_str = ','.join(map(str, batch_ids))
    
    # Define the URL for the current batch
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
        f"eventIds={event_ids_str}&"
        "includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&"
        "includePriceHistory=false&includeCommentary=true&includeIncidents=false&includeRace=false&"
        "includePools=false&includeNonFixedOdds=false&lang=nl-NL&channel=I"
    )
    
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
        'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
    }
    
    # Send request
    response = requests.get(url, headers=headers)
    json_data = response.json()

    # Initialize lists to store the data for this batch
    event_ids = []
    event_names = []
    market_names = []
    outcome_names = []
    odds_decimal = []
    prices_numerator = []
    prices_denominator = []
    outcome_types = []
    outcome_subtypes = []

    # Process the JSON response for each event
    for event in json_data['data']['events']:  # Iterate over each event
        event_id = event['id']
        event_name = event['name']
        
        for market in event['markets']:  # Iterate over each market in the event
            market_name = market['name']
            
            for outcome in market['outcomes']:  # Iterate over outcomes for each market
                outcome_name = outcome['name']
                odds_decimal_value = outcome['prices'][0]['decimal']
                numerator = outcome['prices'][0]['numerator']
                denominator = outcome['prices'][0]['denominator']
                outcome_type = outcome['type']
                outcome_subtype = outcome['subType']
                
                # Append captured values to the respective lists
                event_ids.append(event_id)
                event_names.append(event_name)
                market_names.append(market_name)
                outcome_names.append(outcome_name)
                odds_decimal.append(odds_decimal_value)
                prices_numerator.append(numerator)
                prices_denominator.append(denominator)
                outcome_types.append(outcome_type)
                outcome_subtypes.append(outcome_subtype)

    # Create a temporary DataFrame for this batch and append it to all_data
    temp_df = pd.DataFrame({
        'event_id': event_ids,
        'Event Name': event_names,
        'Market Name': market_names,
        'Outcome Name': outcome_names,
        'Odds (Decimal)': odds_decimal,
        'Price Numerator': prices_numerator,
        'Price Denominator': prices_denominator,
        'Outcome Type': outcome_types,
        'Outcome SubType': outcome_subtypes
    })
    
    all_data.append(temp_df)
    
    # Wait between 1 and 3 seconds
    time.sleep(random.uniform(1, 3.001))

# Concatenate all individual DataFrames into a final DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Show the final DataFrame
print(final_df)

      event_id                         Event Name  \
0      5943214  Daniil Medvedev vs Alex De Minaur   
1      5943214  Daniil Medvedev vs Alex De Minaur   
2      5943214  Daniil Medvedev vs Alex De Minaur   
3      5943214  Daniil Medvedev vs Alex De Minaur   
4      5943214  Daniil Medvedev vs Alex De Minaur   
...        ...                                ...   
48814  5887010      Mickey Gall vs Ramiz Brahimaj   
48815  5896181    Mauricio Ruffy vs James Llontop   
48816  5896181    Mauricio Ruffy vs James Llontop   
48817  5896181    Mauricio Ruffy vs James Llontop   
48818  5896181    Mauricio Ruffy vs James Llontop   

                          Market Name     Outcome Name  Odds (Decimal)  \
0         Alex De Minaur Wint een Set               Ja            1.35   
1         Alex De Minaur Wint een Set              Nee            2.95   
2                           Wedstrijd   Alex De Minaur            1.93   
3                           Wedstrijd  Daniil Medvedev            1

In [9]:
final_df

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType
0,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Ja,1.35,7,20,--,None
1,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Nee,2.95,39,20,--,None
2,5943214,Daniil Medvedev vs Alex De Minaur,Wedstrijd,Alex De Minaur,1.93,93,100,HH,A
3,5943214,Daniil Medvedev vs Alex De Minaur,Wedstrijd,Daniil Medvedev,1.92,23,25,HH,H
4,5943214,Daniil Medvedev vs Alex De Minaur,Aantal Games - Odd/Even,Odd,1.87,87,100,OE,1
...,...,...,...,...,...,...,...,...,...
48814,5887010,Mickey Gall vs Ramiz Brahimaj,Aantal Rondes - Over/Under 2.5,Over,1.57,57,100,HL,H
48815,5896181,Mauricio Ruffy vs James Llontop,Winnaar Gevecht,James Llontop,6.00,5,1,HH,A
48816,5896181,Mauricio Ruffy vs James Llontop,Winnaar Gevecht,Mauricio Ruffy,1.11,11,100,HH,H
48817,5896181,Mauricio Ruffy vs James Llontop,Aantal Rondes - Over/Under 1.5,Over,1.59,59,100,HL,H


In [10]:
# Perform the left join on 'event_id' to get 'sport' and 'group_name' from events_df
final_df = final_df.merge(final_matches_df[['event_id', 'sport', 'competition']], on='event_id', how='left').drop_duplicates()

In [11]:
# Replace 'A' with 2 and 'H' with 1 in the outcome_subtype column
final_df['Outcome SubType'] = final_df['Outcome SubType'].replace({'A': '2', 'H': '1'})

# Display the updated final_df
print(final_df)

      event_id                         Event Name  \
0      5943214  Daniil Medvedev vs Alex De Minaur   
1      5943214  Daniil Medvedev vs Alex De Minaur   
2      5943214  Daniil Medvedev vs Alex De Minaur   
3      5943214  Daniil Medvedev vs Alex De Minaur   
4      5943214  Daniil Medvedev vs Alex De Minaur   
...        ...                                ...   
98988  5887010      Mickey Gall vs Ramiz Brahimaj   
98989  5896181    Mauricio Ruffy vs James Llontop   
98990  5896181    Mauricio Ruffy vs James Llontop   
98991  5896181    Mauricio Ruffy vs James Llontop   
98992  5896181    Mauricio Ruffy vs James Llontop   

                          Market Name    Outcome Name  Odds (Decimal)  \
0         Alex De Minaur Wint een Set              Ja            1.35   
1         Alex De Minaur Wint een Set              Ja            1.35   
2         Alex De Minaur Wint een Set              Ja            1.35   
3         Alex De Minaur Wint een Set             Nee            2.95  

In [12]:
final_df[final_df['Market Name'].str.contains('Draw')]

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition
261,5958355,Estudiantes de La Plata (W) vs CA San Lorenzo ...,Draw No Bet,Estudiantes de La Plata (W),4.00,3,1,DN,1,Voetbal,Argentina Campeonato Femenino
262,5958355,Estudiantes de La Plata (W) vs CA San Lorenzo ...,Draw No Bet,Estudiantes de La Plata (W),4.00,3,1,DN,1,Voetbal,Argentina Campeonato Femenino
263,5958355,Estudiantes de La Plata (W) vs CA San Lorenzo ...,Draw No Bet,Estudiantes de La Plata (W),4.00,3,1,DN,1,Voetbal,Argentina Campeonato Femenino
264,5958355,Estudiantes de La Plata (W) vs CA San Lorenzo ...,Draw No Bet,CA San Lorenzo De Almagro (W),1.14,7,50,DN,2,Voetbal,Argentina Campeonato Femenino
265,5958355,Estudiantes de La Plata (W) vs CA San Lorenzo ...,Draw No Bet,CA San Lorenzo De Almagro (W),1.14,7,50,DN,2,Voetbal,Argentina Campeonato Femenino
...,...,...,...,...,...,...,...,...,...,...,...
98498,5927585,Radnicki Kraguljevac vs RK Vranje 1957,Draw No Bet,RK Vranje 1957,8.00,7,1,DN,2,Handbal,Serbia Superliga
98499,5927585,Radnicki Kraguljevac vs RK Vranje 1957,Draw No Bet,Radnicki Kraguljevac,1.03,3,100,DN,1,Handbal,Serbia Superliga
98500,5927585,Radnicki Kraguljevac vs RK Vranje 1957,Draw No Bet,Radnicki Kraguljevac,1.03,3,100,DN,1,Handbal,Serbia Superliga
98952,5578447,Orebro HK vs Rogle BK,Draw No Bet,Rogle BK,2.05,21,20,DN,2,IJshockey,Zweden SHL


In [13]:
final_df[final_df['Event Name'].str.contains('Minaur')]

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition
0,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Ja,1.35,7,20,--,None,Tennis,"ATP Finals, Enkelspel"
1,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Ja,1.35,7,20,--,None,Tennis,"ATP Finals, Enkelspel"
2,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Ja,1.35,7,20,--,None,Tennis,"ATP Finals, Enkelspel"
3,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Nee,2.95,39,20,--,None,Tennis,"ATP Finals, Enkelspel"
4,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Nee,2.95,39,20,--,None,Tennis,"ATP Finals, Enkelspel"
...,...,...,...,...,...,...,...,...,...,...,...
91338,5943214,Daniil Medvedev vs Alex De Minaur,Game - Handicap Winnaar 0.5,Daniil Medvedev,1.79,79,100,WH,1,Tennis,"ATP Finals, Enkelspel"
91339,5943214,Daniil Medvedev vs Alex De Minaur,Game - Handicap Winnaar 0.5,Daniil Medvedev,1.79,79,100,WH,1,Tennis,"ATP Finals, Enkelspel"
91340,5943214,Daniil Medvedev vs Alex De Minaur,Price Boost,Medvedev wint en meer dan 22 games in de wedst...,3.75,11,4,--,None,Tennis,"ATP Finals, Enkelspel"
91341,5943214,Daniil Medvedev vs Alex De Minaur,Price Boost,Medvedev wint en meer dan 22 games in de wedst...,3.75,11,4,--,None,Tennis,"ATP Finals, Enkelspel"


In [14]:
[i for i in list(set(final_df['Market Name'])) if 'No Bet' in i]

['2e Helft: Draw No Bet', 'Draw No Bet', '1e Helft: Draw No Bet']

In [17]:
final_df.to_csv('totoAllSports.csv')

In [15]:
# # Get all offers for all matches of Football event_ids
# # Initialize an empty list to accumulate all data
# all_data = []

# # Loop over all event_ids in your matches_df
# for event_id in final_matches_df['event_id']:
#     url = f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?eventIds={event_id}&includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&includePriceHistory=false&includeCommentary=true&includeIncidents=false&includeRace=false&includePools=false&includeNonFixedOdds=false&lang=nl-NL&channel=I"

#     payload = {}
#     headers = {
#         'accept': 'application/json',
#         'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
#         'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
#     }

#     response = requests.get(url, headers=headers, data=payload)
#     json_data = json.loads(response.text)

#     # Initialize empty lists to store the data for this event
#     event_ids = []
#     event_names = []
#     market_names = []
#     outcome_names = []
#     odds_decimal = []
#     prices_numerator = []
#     prices_denominator = []
#     outcome_types = []
#     outcome_subtypes = []

#     # Iterate over the events, markets, and outcomes
#     for event in json_data['data']['events']:  # Iterate over each event
#         event_id = event['id']
#         event_name = event['name']
        
#         for market in event['markets']:  # Iterate over each market in the event
#             market_name = market['name']
            
#             for outcome in market['outcomes']:  # Iterate over outcomes for each market
#                 outcome_name = outcome['name']
#                 odds_decimal_value = outcome['prices'][0]['decimal']
#                 numerator = outcome['prices'][0]['numerator']
#                 denominator = outcome['prices'][0]['denominator']
#                 outcome_type = outcome['type']
#                 outcome_subtype = outcome['subType']
                
#                 # Append captured values to the respective lists
#                 event_ids.append(event_id)
#                 event_names.append(event_name)
#                 market_names.append(market_name)
#                 outcome_names.append(outcome_name)
#                 odds_decimal.append(odds_decimal_value)
#                 prices_numerator.append(numerator)
#                 prices_denominator.append(denominator)
#                 outcome_types.append(outcome_type)
#                 outcome_subtypes.append(outcome_subtype)

#     # Create a temporary DataFrame for this batch
#     temp_df = pd.DataFrame({
#         'Event ID': event_ids,
#         'Event Name': event_names,
#         'Market Name': market_names,
#         'Outcome Name': outcome_names,
#         'Odds (Decimal)': odds_decimal,
#         'Price Numerator': prices_numerator,
#         'Price Denominator': prices_denominator,
#         'Outcome Type': outcome_types,
#         'Outcome SubType': outcome_subtypes
#     })
    
#     # Append the temporary DataFrame to the all_data list
#     all_data.append(temp_df)
#     time.sleep(random.uniform(1, 3.001))

# # Concatenate all individual DataFrames into a final DataFrame
# final_df = pd.concat(all_data, ignore_index=True)

# # Show the final DataFrame
# print(final_df)

In [16]:
# Get all bets for 1 match

# import pandas as pd

# # Initialize empty lists to store the data
# event_ids = []
# event_names = []
# market_names = []
# outcome_names = []
# odds_decimal = []
# prices_numerator = []
# prices_denominator = []
# outcome_types = []
# outcome_subtypes = []

# # Iterate over the events, markets, and outcomes
# for event in json_data['data']['events']:  # Iterate over each event
#     event_id = event['id']
#     event_name = event['name']
    
#     for market in event['markets']:  # Iterate over each market in the event
#         market_name = market['name']
        
#         for outcome in market['outcomes']:  # Iterate over outcomes for each market
#             outcome_name = outcome['name']
#             odds_decimal_value = outcome['prices'][0]['decimal']
#             numerator = outcome['prices'][0]['numerator']
#             denominator = outcome['prices'][0]['denominator']
#             outcome_type = outcome['type']
#             outcome_subtype = outcome['subType']
            
#             # Append captured values to the respective lists
#             event_ids.append(event_id)
#             event_names.append(event_name)
#             market_names.append(market_name)
#             outcome_names.append(outcome_name)
#             odds_decimal.append(odds_decimal_value)
#             prices_numerator.append(numerator)
#             prices_denominator.append(denominator)
#             outcome_types.append(outcome_type)
#             outcome_subtypes.append(outcome_subtype)

# # Create a DataFrame with the collected data
# df = pd.DataFrame({
#     'Event ID': event_ids,
#     'Event Name': event_names,
#     'Market Name': market_names,
#     'Outcome Name': outcome_names,
#     'Odds (Decimal)': odds_decimal,
#     'Price Numerator': prices_numerator,
#     'Price Denominator': prices_denominator,
#     'Outcome Type': outcome_types,
#     'Outcome SubType': outcome_subtypes
# })

# # Show the DataFrame
# df